In [1]:
import os

In [2]:
%pwd

'd:\\Manav\\Projects\\Wine-Quality-Prediction\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\Manav\\Projects\\Wine-Quality-Prediction'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str

In [10]:
from wineQuality.constants import *
from wineQuality.utils import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(self,
                config_filepath= CONFIG_FILE_PATH,
                params_filepath= PARAMS_FILE_PATH,
                schemas_filepath= SCHEMAS_FILE_PATH):

                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schemas = read_yaml(schemas_filepath)
                
                create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schemas.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            metric_file_name= config.metric_file_name,
            all_params= params,
            target_column= schema.name
        )
        
        return model_evaluation_config

In [9]:
import os
import pandas as pd 
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [11]:
class ModelEvaluation:
    def __init__(self,config=ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2
    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        X_test = test_data.drop([self.config.target_column],axis=1)
        y_test = test_data[[self.config.target_column]]
        
        preds = model.predict(X_test)
        
        (rmse, mae, r2) = self.eval_metrics(y_test,preds)
        
        #Saving Metrics results
        scores = {'rmse':rmse, 'mae':mae, 'r2':r2}
        
        save_json(path=Path(self.config.metric_file_name),data=scores)

In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.eval_metrics()
    model_evaluation_config.save_results()
except Exception as e:
    raise e